# 手机价格分类
## 导入所需的包与类

In [1]:
import math
import pandas
import functools
import time
from sklearn.svm import SVC

## 工具函数
### 数据集切分
函数`split(data, train, valid, test)`按照`train`:`valid`:`test`的比例将表格`data`横向切分为三份,
并返回这三份表格.

In [2]:
def split(data, train, valid, test):
    # calculate the size of each part
    size = len(data)
    size_train = int(size * train / (train + valid + test))
    size_valid = int(size * valid / (train + valid + test))

    # split the data
    data_train = data[0:size_train]
    data_valid = data[size_train:size_train + size_valid]
    data_test = data[size_train + size_valid:size]

    return data_train, data_valid, data_test

### 分离属性与标记
函数`get_xs_ys(data)`将表格`data`中标记一列(price_range)与其他列分离开来,
并返回分离后的两部分.

In [3]:
def get_xs_ys(data):
    xs = data.drop('price_range', axis=1).values.tolist()
    ys = data['price_range'].values.tolist()
    return xs, ys

### 计算分类错误率
函数`score(model, xs, ys)`用模型`model`对一组样本`xs`进行预测,
并将预测结果与相应的一组参考标记`ys`进行比对, 计算出预测错误率.

In [4]:
def score(model, xs, ys):
    # conduct the prediction
    predicted_ys = model.predict(xs)

    # calculate the error rate
    return sum([y != predicted_y for y, predicted_y in zip(ys, predicted_ys)]) / len(xs)

### 驱动函数
函数`run(preprocess, model_class, data)`依次执行预处理, 数据集切分, 模型训练与预测错误率评估,
然后返回测试集上的预测错误率, 训练集上的预测错误率与模型训练所用时间(单位: 秒).
所用预处理函数由参数`preprocessor`指定, 所用学习算法由参数`model_class`指定,
所用数据由参数`data`指定.

In [5]:
def run(preprocess, model_class, data):
    # preprocess
    data = preprocess(data)

    # split the data into training set, validation set and test set
    data_train, data_valid, data_test = split(data, 8, 1, 1)

    # split attributes and label
    xs_train, ys_train = get_xs_ys(data_train)
    xs_valid, ys_valid = get_xs_ys(data_valid)
    xs_test, ys_test = get_xs_ys(data_test)

    # train a model
    model = model_class()
    time_start = time.time()
    model.fit(xs_train, ys_train)
    time_finish = time.time()

    # calculate the error rates and the time used to train the model
    score_test = score(model, xs_test, ys_test)
    score_train = score(model, xs_train, ys_train)
    time_cost = time_finish - time_start

    return score_test, score_train, time_cost

## 预处理函数与模型类

对于每个学习算法, 都有一个预处理函数与一个模型类.

预处理函数`preprocess(data)`对表格`data`中的原始数据进行预处理,
然后返回处理后的数据表格.

模型类有两个成员方法. 第一个成员方法是`fit(self, xs, ys)`,
它会使用一组样本`xs`与对应的一组标记`ys`来训练模型, 并将训练得到的模型存放在`self`对象中.
第二个成员方法是`predict(self, xs)`, 它会使用`self`对象中的模型来对一组样本`xs`进行预测,
然后返回预测结果(一组标记).

### 逻辑回归

逻辑回归的预处理函数为`preprocess_logistic`. 在预处理过程中,
原始数据中每个属性值都调整到$[0, 1]$区间中, 这样做可以避免模型训练过程中出现数据溢出的问题.

In [6]:
def preprocess_logistic(data):
    # avoid to change the original DataFrame object
    data = data.copy()

    # normalize data
    for col_name in data.columns:
        if col_name != 'price_range':
            s = data[col_name].max()
            data[col_name] = data[col_name].apply(lambda x: x / s)

    return data

逻辑回归的模型类为`LogisticRegression`. 成员变量`ws`表示模型对应的$logit$函数的参数向量$(w_0, w_1, \ldots, w_d)$.
私有成员方法`_calc_post_prob(self, x)`使用`self`对象中的模型计算样本`x`属于正类的后验概率.

In [7]:
class LogisticRegression:
    def __init__(self):
        self.ws = [] # params of logit function

    # train a model
    def fit(self, xs, ys):
        # define constants
        eta = 0.001
        conv_bound = 0.003 # convergence bound

        # calculate number of attributes
        attr_num = len(xs[0]) + 1

        # initialize params of logit function
        self.ws = [0.0] * attr_num

        # learn the params with gradient-descent algorithm
        while True:
            dws = [0.0] * attr_num
            for x, y in zip(xs, ys):
                post_prob = self._calc_post_prob(x)
                dws = [dw + (y - post_prob) * attr for dw, attr in zip(dws, [1.0, *x])]
            if all([abs(eta * dw) < conv_bound for dw in dws]):
                break
            self.ws = [w + eta * dw for w, dw in zip(self.ws, dws)]

    # conduct prediction
    def predict(self, xs):
        return [int(self._calc_post_prob(x) > 0.5) for x in xs]

    # calculate posterior possibility
    def _calc_post_prob(self, x):
        a = sum([w * attr for w, attr in zip(self.ws, [1.0, *x])])
        return 1 / (1 + math.exp(-a))

### 朴素贝叶斯分类器

朴素贝叶斯分类器的预处理函数为`preprocess_bayesian`. 原始数据中,
有一些属性有很多可能的取值, 使得模型在预测时难以用训练数据估计这些属性上的似然.
在预处理过程中, 对于这些属性, 属性取值范围被划分为一组区间, 每个属性值被其所属的区间的编号所替代.

In [8]:
def preprocess_bayesian(data):
    # avoid to change the original DataFrame object
    data = data.copy()

    # discretize continuous attributes into intervals
    # split large numbers into ranges
    data['battery_power'] = pandas.cut(data['battery_power'], 10, labels=False)
    data['clock_speed'] = pandas.cut(data['clock_speed'], 10, labels=False)
    data['fc'] = pandas.cut(data['fc'], 10, labels=False)
    data['int_memory'] = pandas.cut(data['int_memory'], 10, labels=False)
    data['mobile_wt'] = pandas.cut(data['mobile_wt'], 10, labels=False)
    data['px_height'] = pandas.cut(data['px_height'], 10, labels=False)
    data['px_width'] = pandas.cut(data['px_width'], 10, labels=False)
    data['ram'] = pandas.cut(data['ram'], 10, labels=False)

    return data

朴素贝叶斯分类器的模型类为`NaiveBayesianClassifier`.
成员变量`tables`的意义: `tables[c][i][x]`记录了训练数据中属于第$c$类且第$i$个属性取值为$x$的样例个数.
成员变量`ns`记录了训练数据中正类与反类的样例个数. 模型的训练过程即为统计得到这两个成员变量的过程.

In [9]:
class NaiveBayesianClassifier:
    def __init__(self):
        self.tables = [[], []] # tables[c][i][x]: number of examples for which C = c and x_i = x
        self.ns = [0, 0] # number of negative/positive examples

    # train a model
    def fit(self, xs, ys):
        # calculate number of attributes
        attr_num = len(xs[0])

        # count
        for c in [0, 1]:
            self.ns[c] = ys.count(c)
            self.tables[c] = [dict() for _ in range(attr_num)]
            for x in [x for x, y in zip(xs, ys) if y == c]:
                for attr, row in zip(x, self.tables[c]):
                    row.setdefault(attr, 0)
                    row[attr] += 1

    # conduct prediction
    def predict(self, xs):
        # calculate number of attributes
        attr_num = len(xs[0])

        # calculate the prior probabilities
        total = sum(self.ns)
        prior_probs = [self.ns[c] / total for c in [0, 1]]

        # conduct the prediction
        ys = []
        for x in xs:
            f = [0.0, 0.0]
            for c in [0, 1]:
                # calculate the likelihood of each attribute
                likelihoods = [self.tables[c][i].get(x[i], 0) / self.ns[c] for i in range(attr_num)]
                # calculate the decision function value
                f[c] = functools.reduce(lambda v, e: v * e, [prior_probs, *likelihoods])
            # take the class with the max decision function value
            ys.append(f.index(max(f)))

        return ys

### 支持向量机

支持向量机的预处理函数为`preprocess_svm`. 预处理过程没有对原始数据做修改.

In [10]:
def preprocess_svm(data):
    # avoid to change the original DataFrame object
    data = data.copy()

    return data

支持向量机的模型类为`SVM`. 该类的实现借助了`sklearn.svm.SVC`类.

In [11]:
class SVM:
    def __init__(self):
        self.svc = SVC() # sklearn model

    # train a model
    def fit(self, xs, ys):
        # train the sklearn model
        self.svc.fit(xs, ys)

    # conduct prediction
    def predict(self, xs):
        # predict with the sklearn model
        return self.svc.predict(xs)